In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from google.colab import drive
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from matplotlib import pyplot as plt

In [ ]:
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/dataset.csv')
df.head()

Mounted at /content/drive


,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [ ]:
#single feature model (linear regrssion)


In [ ]:
def categorize_popularity(popularity):
       if 0 <= popularity <= 50:
           return 0
       elif 51 <= popularity <= 100:
           return 1
       else:
           return -1  # Handle values outside the range if necessary

df['popularity_category'] = df['popularity'].apply(categorize_popularity)
new_df = df[['duration_ms',	'danceability', 'energy','loudness','speechiness','acousticness','instrumentalness','liveness','valence','tempo','time_signature', 'popularity_category']]  # Select desired columns
new_df.corr()

#loudness and energy correlate some correlation between valance and danceability
#acousticness and energy/loudness have a negative correlation

,duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,popularity_category
duration_ms,1.000000,-0.073426,0.058523,-0.003470,-0.062600,-0.103788,0.124371,0.010321,-0.154479,0.024346,0.018225,-0.040963
danceability,-0.073426,1.000000,0.134325,0.259077,0.108626,-0.171533,-0.185606,-0.131617,0.477341,-0.050450,0.207218,0.052042
energy,0.058523,0.134325,1.000000,0.761690,0.142509,-0.733906,-0.181879,0.184796,0.258934,0.247851,0.187126,-0.022272
loudness,-0.003470,0.259077,0.761690,1.000000,0.060826,-0.589803,-0.433477,0.076899,0.279848,0.212446,0.191992,0.034287
speechiness,-0.062600,0.108626,0.142509,0.060826,1.000000,-0.002186,-0.089616,0.205219,0.036635,0.017273,-0.000011,-0.049382
acousticness,-0.103788,-0.171533,-0.733906,-0.589803,-0.002186,1.000000,0.104027,-0.020700,-0.107070,-0.208224,-0.176138,-0.035406
instrumentalness,0.124371,-0.185606,-0.181879,-0.433477,-0.089616,0.104027,1.000000,-0.079893,-0.324312,-0.050330,-0.082580,-0.081798
liveness,0.010321,-0.131617,0.184796,0.076899,0.205219,-0.020700,-0.079893,1.000000,0.019086,0.000600,-0.023651,-0.086904
valence,-0.154479,0.477341,0.258934,0.279848,0.036635,-0.107070,-0.324312,0.019086,1.000000,0.078273,0.133686,-0.040244
tempo,0.024346,-0.050450,0.247851,0.212446,0.017273,-0.208224,-0.050330,0.000600,0.078273,1.000000,0.066641,-0.013160


In [ ]:
#multi feature model (linear regression)

In [ ]:
#other calssificaion things(if we want to)